In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/wrime.tsv", sep="\t")

df["Readers_mean"] = df[["Reader1_Joy", "Reader2_Joy", "Reader3_Joy"]].mean(axis=1)
df["Readers_std"] = df[["Reader1_Joy", "Reader2_Joy", "Reader3_Joy"]].std(axis=1)
df["Difference"] = df["Writer_Joy"] - df["Readers_mean"]

df["Label"] = 0  # 客観3人のばらつきが大きい（標準偏差が1以上）
df.loc[df["Readers_std"] < 1, "Label"] = 1  # 主観と客観の差が少ない（差の絶対値が1以下）
df.loc[(df["Readers_std"] < 1) & (df["Difference"] > 1), "Label"] = 2 # 主観 > 客観
df.loc[(df["Readers_std"] < 1) & (df["Difference"] < -1), "Label"] = 3 # 主観 < 客観

df = df[
    [
        "Sentence",
        "Writer_Joy",
        "Reader1_Joy",
        "Reader2_Joy",
        "Reader3_Joy",
        "Readers_mean",
        "Readers_std",
        "Difference",
        "Label",
    ]
]

train_df, test_df = train_test_split(df, test_size=0.20, random_state=0)

print("訓練用データ", len(train_df))
print("評価用データ", len(test_df))

train_df.to_csv("../data/train.tsv", sep="\t", index=False)
test_df.to_csv("../data/test.tsv", sep="\t", index=False)

訓練用データ 34560
評価用データ 8640


In [2]:
pd.DataFrame(
    {
        "train": train_df["Label"].value_counts().sort_index(),
        "test": test_df["Label"].value_counts().sort_index(),
        "all": df["Label"].value_counts().sort_index(),
    }
)

,train,test,all
0,5303,1338,6641
1,25331,6322,31653
2,3502,884,4386
3,424,96,520
